### 实现反向传播的具体操作
现在我们只考虑一个简单的神经网络，它只有一个隐藏层和一个输出节点。这是通过反向传播
更新权重的算法概述：  
- 把每一层权重更新的初始步长设置为0
> - 输入到隐藏层的权重更新是$\Delta w_{ij}=0$
> - 隐藏层到输出层的权重更新是$\Delta W_j = 0$
  
  
- 对训练数据当中的每一个点
> - 让它正向通过网络，计算输出$\hat{y}$
> - 计算输出节点的误差梯度 $\delta^o =(y-\hat{y})\hat{y}(1-\hat{y})$,这里
$\hat{y}=f(z),z=\sum_j W_j bj$是输出节点的输入。
> - 误差传播到隐藏层 $\delta_j^h=\delta^o W_j b_j(1-b_j)$,这里$b_j=f(\sum_i w_{ij}x_i)$
> - 更新权重步长：
> >  $\Delta W_j=\Delta W_j +\delta^0 b_j$  
> >  $\Delta w_{ij}=\Delta w_{ij}+\delta_j^h x_i$  
- 更新权重，其中 $\eta$是学习率，m是数据点的数量：  
> - $W_j=W_j+\eta\Delta W_j/m$
> - $w_{ij}=w_{ij}+\eta w_{ij}/m$
- 重复这个过程e代

### 注意：
首先你需要初始化权重。我们希望它们比较小，这样输入在 sigmoid 函数那里可以在接近 0 的位置，而不是最高或者最低处。  
很重要的一点是要随机地初始化它们，这样它们有不同的初始值，是发散且不对称的。  
所以我们用一个中心为 0 的正态分布来初始化权重，此正态分布的标准差（scale 参数）最好使用 $1/\sqrt{n}$,
其中n是输入单元的个数。这样就算是输入单元的数量变多，sigmoid 的输入还能保持比较小。

In [ ]:
import numpy as np
from data_prep import features, targets, features_test, targets_test
np.random.seed(21)

def sigmoid(x):
    return 1/(1+np.exp(-x))

n_hidden=2
epochs=900
learnrate=0.005

n_records, n_features= features.shape
last_loss=None
# Initialize weights
weights_input_hidden=np.random.normal(scale=1/(n_features**0.5),
                                      size=(n_features,n_hidden))
weights_hidden_output=np.random.normal(scale=1/(n_features**0.5),
                                      size=n_hidden)
for e in range(epochs):
    del_w_input_hidden=np.zeros(weights_input_hidden.shape)
    del_w_hidden_output=np.zeros(weights_hidden_output.shape)
    
    for x,y in zip(features,targets):
        # calculate output
        hidden_layer_input=np.dot(x,weights_input_hidden)
        hidden_layer_output=sigmoid(hidden_layer_input)
        
        output_layer_in=np.dot(hidden_layer_output,weights_hidden_output)
        output=sigmoid(output_layer_in)
        
        # calculate error term for the output unit
        
        error=y-output
        output_error_term=error*output*(1-output)
        
        #propagate errors to hidden layer
        
        # calculate the error term for the hidden layer
        hidden_error=output_error_term*weights_hidden_output
        hidden_error_term=hidden_error*hidden_layer_output*(1-hidden_layer_output)
        # update the change in weights
        del_w_input_hidden += hidden_error_term*x[:,None]
        del_w_hidden_output += output_error_term*hidden_layer_output
        #update weights
    weights_input_hidden += learnrate*del_w_input_hidden/n_records
    weights_hidden_output += learnrate*del_w_hidden_output/n_records
        
        #print out mean square error on training set
    if e%(epochs/10) == 0:
        hidden_output=sigmoid(np.dot(x,weights_input_to_hidden))
        out=sigmoid(np.dot(hidden_output,weights_hidden_to_output))
            
        loss=np.mean((out-targets)**2)
            
        if last_loss and last_loss < loss:
            print('Train loss:', loss,'Warning-Loss Increasing')
        else:
            print('Train loss:',loss)
        last_loss = loss
#calculate accuracy on test data
hidden = sigmoid(np.dot(feature_test, weights_input_hidden))
out= sigmoid(np.dot(hidden,weights_hidden_output))
predictions = out >0.5
accuracy=np.mean(predictions == targets_test) 
print('Predictions accuracy: {:.3f}'.format(accuracy))